SuperChip Optimization Model

We will start by importing the necceesry libraries.

In [1]:
import gurobipy as gp
import pandas as pd

m = gp.Model('Super chip Optimization')

m.modelsense = gp.GRB.MINIMIZE
m.update()


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-16


The next step is creating dataframes from the excel file to enable creation of dictionaries

In [2]:
# Load data from excel file
df = pd.read_excel('SuperChipData.xlsx', sheet_name='Production Costs')
df_ship = pd.read_excel('SuperChipData.xlsx', sheet_name='Shipping Costs')
df_prod_cap = pd.read_excel('SuperChipData.xlsx', sheet_name='Production Capacity')
df_demand = pd.read_excel('SuperChipData.xlsx', sheet_name='Sales Region Demand')


In [3]:
# Creating list of facilities, chips and regions
facilities = df['Facility'].unique()
chips = df['Computer Chip'].unique()
regions = df_ship['Sales Region'].unique()

The next cell is the creation of dictionaries for the formulation of the problen

In [12]:
# Creating production costs dictionary
prod_costs = {(row['Computer Chip'], row['Facility']): row['Production Cost ($ per chip)'] for index, row in df.iterrows()}

# Creating shipping costs dictionary
shipping_costs = {(row['Computer Chip'], row['Facility'], row['Sales Region']): row['Shipping Cost ($ per chip)'] for index, row in df_ship.iterrows()}

# creating production capacity dictionary
prod_cap = {(row['Facility']): row['Computer Chip Production Capacity (thousands per year)']*1000 for index, row in df_prod_cap.iterrows()}

# creating demand dictionary
demand = {(row['Computer Chip'], row['Sales Region']): row['Yearly Demand (thousands)']*1000 for index, row in df_demand.iterrows()}


This dictionary will be used for Question 3

In [5]:
# creating a dictionary for a 10% increase in demand
new_demand = {}
for key, value in demand.items():
    new_demand[key] = value * 1.1


Two decision variable are needed for production and shiping
x_ij represent chip i manufactured in facility j
y_ijk represent chip i shipped from facility j to region k

In [30]:
# Creating decision variable x_ij
x = {}
for i in chips:
    for j in facilities:
        x[i, j] = m.addVar(lb = 0, vtype=gp.GRB.INTEGER, name=f'x{i}_{j}')

        
# Creating decision variable y_ijk
y = {}
for i in chips:
    for j in facilities:
        for k in regions:
            y[i, j, k] = m.addVar(lb = 0, vtype=gp.GRB.INTEGER, name=f'y{i}_{j}_{k}')
            
            
m.update() 

The objective function is the addition of production cost and shipping cost to minimize them.
we didn't want to minimize them seperately.

In [31]:
# Set objective function
production_cost = gp.quicksum(prod_costs[i, j] * x[i, j] for i in chips for j in facilities)
shipping_cost = gp.quicksum(shipping_costs[i, j, k] * y[i, j, k] for i in chips for j in facilities for k in regions)
m.setObjective(production_cost + shipping_cost, gp.GRB.MINIMIZE)
m.update()

In [32]:
# # Set objective function
# m.setObjective(gp.quicksum(prod_costs[i, j] * x[i, j] for i in chips for j in facilities) +
#                gp.quicksum(shipping_costs[i, j, k] * y[i, j, k] for i in chips for j in facilities for k in regions),
#                gp.GRB.MINIMIZE)
# m.update() 

The three contraints represent ensuring the chips in each factory don't exceed the factory capacity, the chips produced equal or exceed the demand of chips, the last contraint ensure the chips shipped is less than the chips produced.

In [33]:
# production capacity constriant
c = {}
for j in facilities:
    c[j] = m.addConstr(gp.quicksum(x[i, j] for i in chips) <= prod_cap[j])

        
# demand constraint
demandConstraint={}
for i in chips:
    for k in regions:
        demandConstraint=m.addConstr(gp.quicksum(y[i, j, k] for j in facilities) >= demand[i, k])

    
#ensuring the chip is produced
shippingConstraint={}
for i in chips:
    for j in facilities:
        shippingConstraint=m.addConstr(gp.quicksum(y[i, j, k] for k in regions) - x[i, j] <= 0)
m.update()

We optimize the LP

In [34]:
m.optimize()
m.write('Super.lp')
m.update()

print ("Optimal Total Cost = ", m.objVal)

# Printing production and shipping costs
print("Optimal Production Cost = ", production_cost.getValue())
print("Optimal Shipping Cost = ", shipping_cost.getValue())


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4225 rows, 14400 columns and 36000 nonzeros
Model fingerprint: 0xe284316f
Variable types: 10800 continuous, 3600 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+05]

MIP start from previous solve produced solution with objective 4.90834e+07 (0.37s)
Loaded MIP start from previous solve with objective 4.90834e+07

Presolve removed 3436 rows and 10860 columns
Presolve time: 0.26s
Presolved: 789 rows, 3540 columns, 7080 nonzeros
Variable types: 3540 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 1211 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth I

This prints the shadow prices for the production capacity contraints(SWITCH vtype to continous)

In [ ]:
df_production = pd.read_excel('chip.xlsx')
# creating a dictionary for our recommended chip_production.
quantity = {(row['Chip'], row['Facility']): row['Value'] for index, row in df_production.iterrows()}

#Creating a dictionary to store the total production cost for each facility

new_cost = {}
for j in facilities:
    for i in chips:
        cost = 0
        if (i,j) in prod_costs.keys():
            costs = prod_costs[(i,j)]
            quat = 0
            for key, value in prod_qty.items():
                if key[1] == j:
                    quat += value
        cost_prod = quat * costs
        cost += cost_prod
        new_cost[j] = cost
        
for j in facilities:
    print("........................................................")
    print(f"The cost of production in {j} facility is {new_cost[j]}")


# Creating a dictionary to store the total production cost for each facility if technology is implemented
reduced_cost = {}
for j in facilities:
    reduced_cost[j] = 0.85 * new_cost[j]

max_key = max(reduced_cost, key=reduced_cost.get)
print("...................................................")
print(f"We Recommended new technology go to {max_key} facility if using the current policy")


new_total_prod_cost = 0
for i,j in quantity:
    if (i,j) in quantity.keys():
        costs = prod_costs[(i,j)]
        cost_prod = quantity[(i,j)] * costs
        new_total_prod_cost += cost_prod
print(".............................................")
print("The new total Production cost from our recommendation is: ", new_total_prod_cost)

# using our recommendation
new_new_cost = {}
for j in facilities:
    for i in chips:
        cost = 0
        if (i,j) in prod_costs.keys():
            costs = prod_costs[(i,j)]
            quat = 0
            for key, value in quantity.items():
                if key[1] == j:
                    quat += value
        cost_prod = quat * costs
        cost += cost_prod
        new_new_cost[j] = cost
        
for j in facilities:
    print("........................................................")
    print(f"The cost of production in {j} facility is {new_new_cost[j]}")


# Creating a dictionary to store the total production cost for each facility if technology is implemented
new_reduced_cost = {}
for j in facilities:
    new_reduced_cost[j] = 0.85 * new_new_cost[j]

new_max_key = max(new_reduced_cost, key=new_reduced_cost.get)
print("...................................................")
print(f"We Recommended new technology go to {new_max_key} facility")

In [ ]:
if m.status == gp.GRB.Status.OPTIMAL:
    for j in facilities:
        print(f'Shadow Price for Production Capacity Constraint {j} = {c[j].Pi}')
else:
    print('No feasible solution found.')

In [ ]:
if m.status == gp.GRB.Status.OPTIMAL:
    for i in chips:
        for j in facilities:
            if x[i,j].x > 0:
                print(f'Number of chip {i} to Produce in {j} Facility: ', x[i, j].x) 
else:
    print('No feasible solution found.')

In [ ]:
if m.status == gp.GRB.Status.OPTIMAL:
    for i in chips:
        for j in facilities:
            for k in regions:
                if x[i,j].x > 0:
                    print(f'Number of chip {i} shipped from {j} Facility to region {k} is : ', y[i, j, k].x) 
else:
    print('No feasible solution found.')

In [ ]:
# Create pandas DataFrame to store decision variable values
x_values_df = pd.DataFrame(columns=['Chip', 'Facility', 'Value'])

# Extract decision variable values and add them to DataFrame
for i in chips:
    for j in facilities:
            x_values_df = x_values_df.append({'Chip': i, 'Facility': j, 'Value': x[i,j].x}, ignore_index=True)

# Write DataFrame to CSV file
x_values_df.to_excel('chip.xlsx', index=False)